<a href="https://colab.research.google.com/github/ochekroun/labs/blob/master/IFAGE_Cours_2_MNIST_avec_Keras%2C_plus_en_d%C3%A9tails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔢 Reconnaitre des chiffres avec Keras

**Deuxième version: avec un réseau de neurones à convolution, et monitorage en utilisant W&B.**


## Installation de Keras

In [ ]:
!pip install -U keras

In [ ]:
# utilisons pytorch plutôt que tensorflow
import os
os.environ["KERAS_BACKEND"] = "torch"

## Exploration du jeu de donnée

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
print("Dimension de x_train:", x_train.shape)
print(x_train.shape[0], "images d'entrainement")
print(x_test.shape[0], "images de test")

In [ ]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [ ]:
from keras.utils import to_categorical

# nombre de classe de notre modèle
num_classes = 10

# on mets les données en forme avec l'encodage "one-hot"
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## Création du modèle

In [ ]:
from keras import layers

In [ ]:
# Réseau de neurones entièrement connecté
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28)),
        layers.Flatten(),
        layers.Dense(200, activation="relu", name="Première couche"),
        layers.Dense(100, activation="relu"),
        layers.Dense(75, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
# Alternative: réseau de neurones à convolution
model = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
# "résumé" de notre modèle
model.summary()

In [ ]:
model.compile(
    loss="categorical_crossentropy", # on veut optimiser la capacité de catégorisation du modèle
    optimizer="adam", # le "moteur" utilisé pour ajuster les paramètres
    metrics=["accuracy"] # ce qu'on veut monitorer: la "précision" du modèle
)

In [ ]:
hist = model.fit(
    x_train, y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.1
)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

## Visualisons la progression de l'entrainement

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.ylim(0.97, 1.0)
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
plot_hist(hist)

## Evaluation sur un example spécifique

In [ ]:
to_test = x_test[1]
plt.imshow(to_test, cmap="gray")

In [ ]:
prediction = model.predict(np.array([to_test]))
plt.bar(range(10), prediction[0], color='blue')
plt.title('Prediction')
plt.xticks(range(10))
plt.xlabel('Digit')
plt.ylabel('Probability')

## Monitorons l'entraînement avec W&B

In [ ]:
!pip install -U wandb

In [ ]:
import wandb
from wandb.keras import WandbMetricsLogger

In [ ]:
wandb.login()

In [ ]:
run = wandb.init(
    project = "keras-mnist",
    config = {
        "batch_size": BATCH_SIZE
    }
)

hist = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=5,
    validation_split=0.1,
    callbacks = [WandbMetricsLogger()]
)

run.finish()